# Clustering

**Manuel Sánchez-Montañés**

First we import the libraries we will need. In addition we will use the first code cell to activate the *inline* mode for the graphics generated by *matplotlib*. We also initialize the seed of the random generator.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Data Load

Now we will use clustering in a wine database. The goal is to check if the clustering discovers the different real wine types.

The database describes the parameters of different wine instances. There are 3 types of wine and 13 wine features with the levels of the most important indicators:
- Alcohol
- Malic acid
- Ash
- Ash alcalinity
- Magnesium
- Total phenols
- Flavanoids
- Nonflavanoid phenols
- Proanthocyanins
- Color intensity
- Hue
- OD280_OD315
- Proline

Now we load this database:


In [ ]:
data = pd.read_csv('./wine_dataset.csv', delimiter=';', header=0)

abro_negrita = '\033[1m'
cierro_negrita = '\033[0m'
print(abro_negrita + 'Wine Database\n' + cierro_negrita )
print('Number of real classes (wine types): %d' % np.unique(data['Type']).shape[0])
print('Unique class labels:', np.unique(data['Type']))
print('\nFirst 5 instances:\n')
data.head(3)

## Data Description

In [ ]:
data.describe()

## Data Exploration

In [ ]:
class_column = 'Type'
classes_names = data['Type'].unique()
attribute_columns = list(data.columns)
attribute_columns.remove(class_column)

print(class_column)
print(classes_names)
print(attribute_columns)

After loading the database we need to do some basic preprocessing: standarization and PCA:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

attribute_columns = list(data.columns.values)
attribute_columns.remove('Type')

print(attribute_columns)

X = data[attribute_columns].values
y = data.Type

In [ ]:
data[attribute_columns].hist(bins=10, figsize=(12,12), xlabelsize=10, ylabelsize=10);

# Clustering with k-means

In [ ]:
clean_data = data[attribute_columns].copy()
attributes_names = attribute_columns

y = np.array(data[class_column])
X = np.array(clean_data[attribute_columns])

Basic preprocessing: standarization and PCA.

In [ ]:
X_std = StandardScaler().fit_transform(X)
X_pca = PCA(n_components=5).fit_transform(X_std)

pd.DataFrame(X_pca).describe()

# IMPORTANT: note that we have used all the patterns in the dataset
# to fit the parameters of StandardScaler and PCA
# This is ONLY ok if we are NOT going to develop a predictive model
# for "y" using this data (in that case, we would use both training and test
# data for fitting the parameters, and using test data for fitting is forbidden!)
#
# If we were going to develop a model for predicting the value
# of a target variable "y" (classification / regression problem),
# first we would need to split the dataset in training and test sets:
#
# testsize = 0.2
# Xaux_train, Xaux_test, y_train, y_test = train_test_split(Xaux, y, test_size=testsize)
#
# and both the normalization and PCA should be fitted using only the training set:
#
# std_scaler = StandardScaler()
# std_scaler.fit(Xaux_train)
# X_std_train = std_scaler.transform(Xaux_train)
# pca = PCA(n_components=2)
# pca.fit(X_std_train)
# X_train = pca.transform(X_std_train)
#
# X_std_test = std_scaler.transform(Xaux_test)
# X_test = pca.transform(X_std_test)


Now we perform k-means searching the optimal number of clusters according to the Calinski-Harabasz score:

In [ ]:
#from sklearn.metrics import silhouette_score as qmetric
from sklearn.metrics import calinski_harabasz_score as qmetric
from sklearn.cluster import KMeans

X_km = X_pca
#X_km = X_std

Nclusters_max = 15
Nrepetitions = 10

qualities = []
inertias = []
models = []
for k in range(1,Nclusters_max+1):
    kmeans = KMeans(n_clusters=k,
                    init='k-means++', n_init=Nrepetitions,
                    max_iter=500, random_state=2)    
    kmeans.fit(X_km)
    models.append(kmeans)
    inertias.append(kmeans.inertia_)
    if k >1:
        qualities.append(qmetric(X_km, kmeans.labels_))
    else:
        qualities.append(0)

In [ ]:
fig = plt.figure(figsize=(14,3))

ax = plt.subplot(1,2,1)
plt.plot(range(1,Nclusters_max+1), inertias, marker='o')
plt.xlabel('number of clusters')
plt.title('Wine database, clustering inertia')

ax = plt.subplot(1,2,2)
plt.plot(range(1,Nclusters_max+1), qualities, marker='o')
plt.xlabel('number of clusters')
plt.title('Wine database, clustering quality')
plt.show()

best = pd.Series(qualities).idxmax() # get index for the best model
kmeans = models[best]
n_clusters = kmeans.get_params()['n_clusters']

In [ ]:
data.shape

In [ ]:
np.unique(kmeans.labels_)

In [ ]:
pcs = [0,1]

cluster_names = ["cluster "+str(c) for c in range(n_clusters)]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
markers = ['s', 'v', 'o', 'd', 's', 'v', 'o', 'd', 's']

fig = plt.figure(figsize=(7,5))
for i in range(n_clusters):
    inds = np.where(kmeans.labels_ == i)[0]
    plt.scatter(X_pca[inds, pcs[0]],
                X_pca[inds, pcs[1]],
                s = 60,
                c = colors[i],
                marker = markers[i],
                alpha = 1.0,
                cmap='RdYlGn',
                label = cluster_names[i])

plt.legend(); plt.grid(); plt.axis('equal'); plt.tight_layout()
plt.title('Credit scoring database, optimal clustering')
plt.xlabel('Principal component '+str(pcs[0]+1))
plt.ylabel('Principal component '+str(pcs[1]+1))
plt.show()

print('Optimal number of clusters:', n_clusters, '\n')

unique_y = np.unique(y)
ids_clusters = kmeans.labels_
for i in np.unique(ids_clusters):
    inds = (np.where(np.array(ids_clusters) == i))[0]
    print('\033[1m'+'- Cluster %d' % i + '\033[0m')
    print('  %g%% of total patterns' % (100*len(inds)/len(ids_clusters)))
    for real_class in unique_y:
        print('  Number of patterns with real class %c: %d' % (real_class, (list(y[inds])).count(real_class)))
    print()

In [ ]:
kmeans.cluster_centers_

In [ ]:
df = pd.DataFrame(data=np.hstack((X,np.reshape(ids_clusters, (len(ids_clusters),1)))),
                  columns=attribute_columns+['cluster_id'])
df.head()

## Interpretation of the obtained clusters

In [ ]:
df.groupby('cluster_id').mean()

In [ ]:
kmeans.cluster_centers_

In [ ]:
sns.set_context("notebook", font_scale=1.1, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(15,n_clusters))
aux = pd.DataFrame(data=np.hstack((X,np.reshape(ids_clusters, (len(ids_clusters),1)))),
                   columns=attribute_columns+['cluster_id'])
df_aux = pd.DataFrame(data=np.hstack((X_std,np.reshape(ids_clusters, (len(ids_clusters),1)))),
                      columns=attribute_columns+['cluster_id'])
aa = sns.heatmap(df_aux.groupby('cluster_id').mean(),
                 annot = aux.groupby('cluster_id').mean(),
                 vmin=-1, vmax=1,
                 cmap='RdYlGn',
                 linewidths=.5, fmt='.2f')
plt.xticks(fontsize=14)
plt.yticks(np.array(range(n_clusters))+0.5,range(n_clusters),fontsize=14)
plt.ylabel('cluster_id', fontsize=14);

In [ ]:
mean_aux = df_aux.groupby('cluster_id').mean().values.T
std_aux  = df_aux.groupby('cluster_id').std().values.T

plt.figure(figsize=(15,5))
for c in range(n_clusters):
    plot_shift = .15/(n_clusters-1)*c
    plt.plot(np.arange(mean_aux.shape[0])+plot_shift,
             mean_aux[:,c], '-o',
             color=colors[c], label=cluster_names[c])
    for i in range(mean_aux.shape[0]):
        plt.plot(2*[i+plot_shift], mean_aux[i,c]+[-std_aux[i,c], std_aux[i,c]],
                 color=colors[c])
plt.xticks(range(mean_aux.shape[0]), attributes_names,
           rotation=90, fontsize=16)
plt.legend(fontsize=12)
plt.yticks([]);

## Interpretation of the clusters using a decision tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from DT_library import tree_to_code, tree_to_pseudo

startbold = '\033[1m'
endbold = '\033[0m'

In [ ]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=10,
                             min_samples_split=20,
                             min_samples_leaf=0.2,
                             #min_weight_fraction_leaf=0.05,
                             #min_impurity_decrease=0.15
                            )

clf = clf.fit(X, ids_clusters)

tree_to_code(clf, attributes_names, start_bold=startbold, end_bold=endbold)

## Obtaining the set of rules equivalent to the tree

In [ ]:
from DT_library import get_rules_from_tree

rules = get_rules_from_tree(clf, attributes_names, cluster_names, X, ids_clusters)

print(len(rules), "rules:\n")
for item in rules.items():
    print('\033[1m' + "* ", item[1][0], '\033[0m')
    for c in item[1][1]:
        print("     "+c[0]+":", c[1], "cases (%.2f%%)" % (100*c[2]))